# LightlyTrain - LTDETR - ONNX and TensorRT Export

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lightly-ai/lightly-train/blob/main/examples/notebooks/object_detection_export.ipynb)

This notebook demonstrates how to export an object detection model to ONNX and TensorRT.

The notebook covers the following steps:
1. Install LightlyTrain
2. Export a trained LTDETR model to ONNX
3. Export a trained LTDETR model to TensorRT
4. Run inference with the TensorRT engine

> **Important**: When running on Google Colab make sure to select a GPU runtime. You can do this by going to `Runtime` > `Change runtime type` and selecting a GPU hardware accelerator.

## Installation

LightlyTrain can be installed directly via `pip`:

In [ ]:
!pip install "lightly-train[onnx,onnxruntime,onnxslim]"

## Export to ONNX

### Load the model weights

Then load the model with LightlyTrain's `load_model` function. This will automatically download the model weights and load the model.

In [ ]:
import lightly_train

model = lightly_train.load_model("dinov3/vitt16-ltdetr-coco")

### Download an example image

Download an example image for inference with the following command:

In [ ]:
!wget -O image.jpg http://images.cocodataset.org/val2017/000000039769.jpg

### Preprocessing

In [ ]:
import torchvision.transforms.v2 as T
from PIL import Image
from torchvision.transforms.functional import pil_to_tensor

# Load image with PIL.
image_pil = Image.open("image.jpg").convert("RGB")

# Convert PIL image to tensor for plotting.
image_tensor = pil_to_tensor(image_pil)

# Define pre-processing transforms.
w, h = image_pil.size
transforms = T.Compose(
    [
        T.Resize((model.image_size)),
        T.ToTensor(),
        T.Normalize(**model.image_normalize) if model.image_normalize else T.Identity(),
    ]
)

# Apply transforms for ONNX and TensorRT inference.
image_tensor_tranformed = transforms(image_pil)[None]

### Get the model predictions for reference

We define a helper function to visualize the predictions.
The function will be used to compare the predictions from PyTorch, ONNX and
TensorRT models.

In [ ]:
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes


def visualize_detections(image, boxes, labels, classes):
    image_with_boxes = draw_bounding_boxes(
        image,
        boxes=boxes,
        labels=[classes[label.item()] for label in labels],
    )
    plt.imshow(image_with_boxes.permute(1, 2, 0))
    plt.axis("off")
    plt.show()

In [ ]:
# Get predictions from the PyTorch model.
labels_torch, boxes_torch, _ = model.predict(image_tensor, threshold=0.6).values()

# Visualize predictions from the PyTorch model.
visualize_detections(
    image_tensor, boxes=boxes_torch, labels=labels_torch, classes=model.classes
)

### Export the model to ONNX

In [ ]:
# Export the PyTorch model to ONNX.
model.export_onnx(out="model.onnx")

In [ ]:
import onnxruntime as ort
import torch

# Create an ONNX Runtime session.
sess = ort.InferenceSession("model.onnx")

labels_onnx, boxes_onnx, scores_onnx = sess.run(
    output_names=None,
    input_feed={
        "images": image_tensor_tranformed.data.numpy(),
    },
)

# The ONNX model does not filter by score, so we do it here.
labels_onnx = labels_onnx[scores_onnx > 0.6]
boxes_onnx = boxes_onnx[scores_onnx > 0.6]

# The ONNX model does not resize boxes to original image size, so we do it here.
scale_x = w / model.image_size[1]
scale_y = h / model.image_size[0]
boxes_onnx[:, 0] *= scale_x  # x1
boxes_onnx[:, 2] *= scale_x  # x2
boxes_onnx[:, 1] *= scale_y  # y1
boxes_onnx[:, 3] *= scale_y  # y2

# Visualize predictions from the ONNX model.
visualize_detections(
    image_tensor,
    boxes=torch.tensor(boxes_onnx).squeeze(0),
    labels=torch.tensor(labels_onnx).squeeze(0),
    classes=model.classes,
)

## Export to TensorRT

**Note:** This notebook requires NVIDIA TensorRT. TensorRT is not part of LightlyTrain’s
dependencies and must be installed separately. Installation depends on your OS, Python
version, GPU, and NVIDIA driver/CUDA setup.
See the [TensorRT documentation](https://docs.nvidia.com/deeplearning/tensorrt/latest/installing-tensorrt/installing.html) for more details.

On CUDA 12.x systems you can often install the Python package via:

In [ ]:
!pip install tensorrt-cu12

In [ ]:
# Get the TensorRT engine.
model.export_tensorrt(
    out="model.trt",
    use_fp16=True,
)

In [ ]:
import numpy as np
import tensorrt as trt


class TRT:
    def __init__(self, engine_path: str, device: str = "cuda:0", verbose: bool = False):
        self.device = torch.device(device)
        self.trt_logger = trt.Logger(trt.Logger.VERBOSE if verbose else trt.Logger.INFO)

        trt.init_libnvinfer_plugins(self.trt_logger, "")
        runtime = trt.Runtime(self.trt_logger)
        with open(engine_path, "rb") as f:
            self.engine = runtime.deserialize_cuda_engine(f.read())
        if self.engine is None:
            raise RuntimeError("Failed to load TensorRT engine.")

        self.context = self.engine.create_execution_context()

        # IO names in engine order.
        self.io_names = [
            self.engine.get_tensor_name(i) for i in range(self.engine.num_io_tensors)
        ]
        self.in_names = [
            n
            for n in self.io_names
            if self.engine.get_tensor_mode(n) == trt.TensorIOMode.INPUT
        ]
        self.out_names = [
            n
            for n in self.io_names
            if self.engine.get_tensor_mode(n) == trt.TensorIOMode.OUTPUT
        ]

        # Allocate buffers once (uses engine's static shapes).
        self.buffers = {}
        self.bindings = []
        for name in self.io_names:
            shape = tuple(self.context.get_tensor_shape(name))
            np_dtype = trt.nptype(self.engine.get_tensor_dtype(name))
            torch_dtype = torch.from_numpy(np.empty((), dtype=np_dtype)).dtype
            t = torch.empty(shape, device=self.device, dtype=torch_dtype)
            self.buffers[name] = t
            self.bindings.append(t.data_ptr())

    @torch.no_grad()
    def __call__(self, inputs: dict[str, torch.Tensor]) -> dict[str, torch.Tensor]:
        # Copy inputs into preallocated buffers.
        for name in self.in_names:
            self.buffers[name].copy_(inputs[name].to(self.device))

        ok = self.context.execute_v2(self.bindings)
        if not ok:
            raise RuntimeError("TensorRT execution failed")

        return {name: self.buffers[name] for name in self.out_names}

In [ ]:
# Instantiate the TensorRT model.
trt_model = TRT("model.trt")

# Run inference with the TensorRT model.
labels_trt, boxes_trt, scores_trt = trt_model(
    {"images": image_tensor_tranformed.to(trt_model.device)}
).values()

# The TensorRT model does not filter by score, so we do it here.
labels_trt = labels_trt[scores_trt > 0.6]
boxes_trt = boxes_trt[scores_trt > 0.6]

# The TensorRT model does not resize boxes to original image size, so we do it here.
scale_x = w / model.image_size[1]
scale_y = h / model.image_size[0]
boxes_trt[:, 0] *= scale_x  # x1
boxes_trt[:, 2] *= scale_x  # x2
boxes_trt[:, 1] *= scale_y  # y1
boxes_trt[:, 3] *= scale_y  # y2

# Visualize predictions from the TensorRT model.
visualize_detections(
    image_tensor,
    boxes=boxes_trt.squeeze(0),
    labels=labels_trt.squeeze(0),
    classes=model.classes,
)